# Credit Risk Resampling Techniques

In [1]:
import warnings


warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from collections import Counter
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import ClusterCentroids





# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('project_cc.csv')
df = pd.read_csv(file_path)
df.tail()

,ID,LIMIT_BAL,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
29995,29996,220000.0,1,39,0,0,0,0,0,0,...,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,29997,150000.0,2,43,-1,-1,-1,-1,0,0,...,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,29998,30000.0,2,37,4,3,2,-1,0,0,...,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,29999,80000.0,1,41,1,-1,0,0,0,-1,...,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1
29999,30000,50000.0,1,46,0,0,0,0,0,0,...,36535.0,32428.0,15313.0,2078.0,1800.0,1430.0,1000.0,1000.0,1000.0,1


In [4]:
#Renaming column
df.rename(columns={'default.payment.next.month':'default_pay'}, inplace=True)

In [5]:
#Adding column for new results
df["New_Results"] = ""
df["Payment_Ratio"] = ""

In [6]:
df.rename(columns={'New_Results':'Total_Missed_Pymts'}, inplace=True)

In [7]:
df['PAY_0'] = [0 if x <= 0 else 1 for x in df['PAY_0']]
df['PAY_2'] = [0 if x <= 0 else 1 for x in df['PAY_2']]
df['PAY_3'] = [0 if x <= 0 else 1 for x in df['PAY_3']]
df['PAY_4'] = [0 if x <= 0 else 1 for x in df['PAY_4']]
df['PAY_5'] = [0 if x <= 0 else 1 for x in df['PAY_5']]
df['PAY_6'] = [0 if x <= 0 else 1 for x in df['PAY_6']]

# 0 = made payment and 1 = missed payment

In [8]:
#Making the Total Missed Payments Column
df['Total_Missed_Pymts']=df['PAY_0']+df['PAY_2']+df['PAY_3']+df['PAY_4']+df['PAY_5']+df['PAY_6']


#Making Total Bill Amt Column
df['Total_Bill_AMT']=df['BILL_AMT1'] + df['BILL_AMT2'] + df['BILL_AMT3'] + df['BILL_AMT4'] + df['BILL_AMT5'] + df['BILL_AMT6']

#Making Total Payment Column
df['Total_Payment']=df['PAY_AMT1'] + df['PAY_AMT2'] + df['PAY_AMT3'] + df['PAY_AMT4'] + df['PAY_AMT5'] + df['PAY_AMT6'] 

#Making % Amount Paid Column
df['%_Amount_Paid'] = df['Total_Payment'] / df['Total_Bill_AMT'] *100


#Making Payment Ratio Column
df['Payment_Ratio']= (df['Total_Missed_Pymts'] / 6) *100




# ***Important***

Notice that the columns "Total Missed Payments" and "Payment Ratio" are now the highest co-related to default. We have successfully created two good features for our data frame!! :) 

In [9]:
df.corr()['default_pay']



ID                   -0.013952
LIMIT_BAL            -0.153520
MARRIAGE             -0.024339
AGE                   0.013890
PAY_0                 0.368117
PAY_2                 0.338022
PAY_3                 0.293583
PAY_4                 0.275495
PAY_5                 0.266959
PAY_6                 0.246089
BILL_AMT1            -0.019644
BILL_AMT2            -0.014193
BILL_AMT3            -0.014076
BILL_AMT4            -0.010156
BILL_AMT5            -0.006760
BILL_AMT6            -0.005372
PAY_AMT1             -0.072929
PAY_AMT2             -0.058579
PAY_AMT3             -0.056250
PAY_AMT4             -0.056827
PAY_AMT5             -0.055124
PAY_AMT6             -0.053183
default_pay           1.000000
Total_Missed_Pymts    0.398394
Payment_Ratio         0.398394
Total_Bill_AMT       -0.012691
Total_Payment        -0.102354
%_Amount_Paid        -0.011391
Name: default_pay, dtype: float64

In [10]:
X=df[['MARRIAGE', 'AGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6','Total_Missed_Pymts', 'Payment_Ratio','Total_Payment']]


In [11]:
X

,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Total_Missed_Pymts,Payment_Ratio,Total_Payment
0,1,24,1,1,0,0,0,0,2,33.333333,689.0
1,2,26,0,1,0,0,0,1,2,33.333333,5000.0
2,2,34,0,0,0,0,0,0,0,0.000000,11018.0
3,1,37,0,0,0,0,0,0,0,0.000000,8388.0
4,1,57,0,0,0,0,0,0,0,0.000000,59049.0
...,...,...,...,...,...,...,...,...,...,...,...
29995,1,39,0,0,0,0,0,0,0,0.000000,42550.0
29996,2,43,0,0,0,0,0,0,0,0.000000,14490.0
29997,2,37,1,1,1,0,0,0,3,50.000000,31300.0
29998,1,41,1,0,0,0,0,0,1,16.666667,147181.0


In [12]:
y=df[['default_pay']]

In [13]:
y.head()

,default_pay
0,1
1,1
2,0
3,0
4,0


# Split the Data into Training and Testing

In [14]:
X.describe()

,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Total_Missed_Pymts,Payment_Ratio,Total_Payment
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04
mean,1.551867,35.485500,0.227267,0.147933,0.140433,0.117000,0.098933,0.102633,0.834200,13.903333,3.165139e+04
std,0.521970,9.217904,0.419073,0.355040,0.347442,0.321426,0.298577,0.303484,1.554303,25.905049,6.082768e+04
min,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,1.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.679750e+03
50%,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.438300e+04
75%,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,16.666667,3.350350e+04
max,3.000000,79.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,100.000000,3.764066e+06


In [15]:
# Check the balance of our target values

y['default_pay'].value_counts()

0    23364
1     6636
Name: default_pay, dtype: int64

In [18]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [19]:
print(f'''% Positive class in Train ={np.round(y_train.value_counts(normalize=True)[1] *100, 2)}''')

% Positive class in Train =default_pay
1              22.08
dtype: float64


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [20]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()



In [21]:
 #Fit the Standard Scaler with the training data
 #When fitting scaling functions, only train on the training dataset
#YOUR CODE HERE
X_scaler = scaler.fit(X_train)


In [22]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Simple Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
lr_y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, lr_y_pred)

0.6170113292082383

In [25]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, lr_y_pred)

array([[5564,  268],
       [1201,  467]], dtype=int64)

In [26]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8071111111111111
Testing Data Score: 0.8041333333333334


In [27]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, lr_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.82      0.95      0.28      0.88      0.52      0.29      5832
          1       0.64      0.28      0.95      0.39      0.52      0.25      1668

avg / total       0.78      0.80      0.43      0.77      0.52      0.28      7500



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [23]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

ros = RandomOverSampler(random_state=1)
# resampling X, y
X_ros, y_ros = ros.fit_resample(X_train_scaled, y_train)
#ros_preds=ros.predict(X_test)
# new class distribution 
print(f''' Shape of X before ROS: { X_train_scaled.shape}
Shape of X after ROS: {X_ros.shape}''')

# View the count of target classes with Counter
# YOUR CODE HERE
Counter(y_ros.value_counts(normalize=True) * 100)

 Shape of X before ROS: (22500, 8)
Shape of X after ROS: (35064, 8)


Counter({50.0: 2})

In [24]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
ros_model= LogisticRegression(solver='lbfgs', random_state=1)
ros_model.fit(X_ros, y_ros)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
ros_y_pred = ros_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, ros_y_pred)

0.6736795748586316

In [26]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, ros_y_pred)

array([[4746, 1086],
       [ 778,  890]], dtype=int64)

In [27]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, ros_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.81      0.53      0.84      0.66      0.45      5832
          1       0.45      0.53      0.81      0.49      0.66      0.42      1668

avg / total       0.77      0.75      0.60      0.76      0.66      0.44      7500



### SMOTE Oversampling

In [28]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE

sm=SMOTE(random_state=1)

Smote_X_resampled, Smote_y_resampled = sm.fit_resample(X_train_scaled, y_train)

print(f''' Shape of X before SMOTE: { X_train_scaled.shape}
Shape of X after SMOTE: {Smote_X_resampled.shape}''')

# View the count of target classes with Counter
# YOUR CODE HERE
Smote_y_resampled.value_counts(normalize=True) *100


 Shape of X before SMOTE: (22500, 8)
Shape of X after SMOTE: (35064, 8)


default_pay
1              50.0
0              50.0
dtype: float64

In [29]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

smote_ros_model= LogisticRegression(solver='lbfgs', random_state=1)
smote_ros_model.fit(Smote_X_resampled, Smote_y_resampled)

LogisticRegression(random_state=1)

In [30]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
smote_y_pred = smote_ros_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, smote_y_pred) 

0.6773248068212097

In [31]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, smote_y_pred)

array([[4806, 1026],
       [ 783,  885]], dtype=int64)

In [32]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, smote_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.82      0.53      0.84      0.66      0.45      5832
          1       0.46      0.53      0.82      0.49      0.66      0.42      1668

avg / total       0.77      0.76      0.60      0.76      0.66      0.44      7500



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
#from imblearn.under_sampling import ClusterCentroids

# Fit the data using `ClusterCentroids` and check the count of each class

cc = ClusterCentroids(random_state=1)
Clust_X_resampled, Clust_y_resampled = cc.fit_resample(X_train_scaled, y_train)



# View the count of target classes with Counter
# YOUR CODE HERE

#Counter(Clust_y_resampled.value_counts)

In [34]:
Counter(y_train.value_counts())

Counter({17532: 1, 4968: 1})

In [35]:
Counter(Clust_y_resampled.value_counts())

Counter({4968: 2})

In [36]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
c_model = LogisticRegression(solver='lbfgs', random_state=1)
c_model.fit(Clust_X_resampled, Clust_y_resampled)

LogisticRegression(random_state=1)

In [37]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
c_y_pred = c_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, c_y_pred)

0.6613838558782603

In [38]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, c_y_pred)

array([[4683, 1149],
       [ 801,  867]], dtype=int64)

In [39]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, c_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.80      0.52      0.83      0.65      0.43      5832
          1       0.43      0.52      0.80      0.47      0.65      0.41      1668

avg / total       0.76      0.74      0.58      0.75      0.65      0.42      7500



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [40]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE

from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
smoteen_X_resampled, smoteen_y_resampled = sm.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE


Counter({6819: 1, 5907: 1})

In [41]:
Counter(y_train.value_counts())

Counter({17532: 1, 4968: 1})

In [42]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
smoteen_model= LogisticRegression(solver='lbfgs', random_state=1)
smoteen_model.fit(smoteen_X_resampled, smoteen_y_resampled)


LogisticRegression(random_state=1)

In [43]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
smoteen_y_pred = smoteen_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, smoteen_y_pred)

0.6739380100857586

In [44]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, smoteen_y_pred)

array([[5249,  583],
       [ 921,  747]], dtype=int64)

In [45]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, smoteen_y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.90      0.45      0.87      0.63      0.42      5832
          1       0.56      0.45      0.90      0.50      0.63      0.38      1668

avg / total       0.79      0.80      0.55      0.79      0.63      0.41      7500



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE:
   
   The Smote and the Naive Random Oversampling models were tied for the best balanced accuracy score.
   

2. Which model had the best recall score?

    YOUR ANSWER HERE.
    
    Every model except the Clustered Centroid and the initial Logistic regression had scores of 99%.

3. Which model had the best geometric mean score?


    YOUR ANSWER HERE.
    
    Other than the Simple Logistic Regression model, every model had the geo score.
